In [1]:
!pip install matplotlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.datasets import cifar100
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, UpSampling2D, Dropout, BatchNormalization, RandomFlip, RandomTranslation, RandomRotation,RandomBrightness, RandomContrast, RandomZoom
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

2024-04-08 21:32:47.039939: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-08 21:32:47.114668: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-08 21:32:48.210008: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
n_epoch = 100
batch_size = 100
taux_validation = 0.1
num_classes = 100
n_images = 50000 # Pour l'entrainement, et 10000 pour le test

In [3]:
def preprocessing(image, label):
  image = tf.image.resize(image, (224, 224)) / 255.0
  label = tf.squeeze(tf.one_hot(label, depth = num_classes), axis = 0)
  return  image, label

augmentation_donnees_keras = Sequential([
    RandomFlip("horizontal"),
    RandomTranslation(0.2,0.2),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomBrightness(0.2,value_range=(0,1)),
    RandomContrast(0.2)
])

def augmentation_donnees(image, label):
  return augmentation_donnees_keras(image), label

train_dataset, test_dataset = cifar100.load_data()

validation_size = int(n_images * taux_validation)
train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset).map(preprocessing).shuffle(n_images, reshuffle_each_iteration=False)
test_dataset = tf.data.Dataset.from_tensor_slices(test_dataset).map(preprocessing)

validation_dataset = train_dataset.take(validation_size).batch(batch_size).map(augmentation_donnees).prefetch(tf.data.AUTOTUNE)
train_dataset = train_dataset.skip(validation_size).batch(batch_size).map(augmentation_donnees).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

2024-04-08 21:32:50.185661: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13290 MB memory:  -> device: 0, name: NVIDIA A2, pci bus id: 0000:b1:00.0, compute capability: 8.6


In [4]:
model = Sequential([
    ResNet50V2(include_top=False, weights='imagenet', pooling="avg"),
    Dropout(0.25),
    Dense(256, activation="sigmoid", kernel_regularizer = tf.keras.regularizers.L1(0.001)),
    Dropout(0.5),
    Dense(num_classes, activation="softmax", kernel_regularizer = tf.keras.regularizers.L2(0.001))
])

In [5]:
%%bash
rm -rf /home/onyxia/modele_enseignant_logs/
mkdir -p /home/onyxia/modele_enseignant_logs/

In [6]:
%load_ext tensorboard

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, start_from_epoch=20)
history = model.fit(train_dataset, epochs=n_epoch, batch_size=batch_size, validation_data = validation_dataset, callbacks = early_stopping_callback)

Epoch 1/100


I0000 00:00:1712612022.061542   41517 service.cc:145] XLA service 0x7f3808014540 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1712612022.061590   41517 service.cc:153]   StreamExecutor device (0): NVIDIA A2, Compute Capability 8.6
2024-04-08 21:33:43.118504: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-08 21:33:46.921275: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900
I0000 00:00:1712612034.059996   42103 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_15955', 108 bytes spill stores, 108 bytes spill loads

I0000 00:00:1712612034.207462   42072 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_23155', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1712612034

KeyboardInterrupt: 

In [19]:
model.save("/home/onyxia/modele_enseignant.keras")

In [21]:
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [22]:
model.evaluate(test_dataset)

100/100 ━━━━━━━━━━━━━━━━━━━━ 42s 414ms/step - accuracy: 0.7075 - loss: 1.9063


[1.8757492303848267, 0.7134000062942505]

In [ ]:
!mc cp ~/modele_enseignant.keras s3/afeldmann/modele_enseignant.keras